# Homework
Your homework is to create a new notebook modeled after this one, where:
- You've put all of your cleaning and prepration steps into a functions you load and use in a module.
- You create and train a simple appropriate model for your task.
- You assess the performance using the appropriate metric for your task.
- Compare the performance with your previous best non Deep Neural Network model. This probably will mean you copy and paste your training from your other notebook into this notebook.

In [1]:
from process import * # module that contains data preprocessing and cleaning (includes libraries)

In [2]:
train = pd.read_csv('/mnt/c/users/efazh/DataProblems/Datasets/Tabular/Zillow/train_2016_v2.csv',parse_dates=["transactiondate"] )
props = pd.read_csv('/mnt/c/users/efazh/DataProblems/Datasets/Tabular/Zillow/property_2016.csv')
sample = pd.read_csv('/mnt/c/users/efazh/DataProblems/Datasets/Tabular/Zillow/sample_submission.csv')

In [3]:
train_df = label_encode(train,props) # label encodes the properties dataset
train_df.head(5)

,parcelid,logerror,transactiondate,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14366692,-0.1684,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12098116,-0.0040,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,12643413,0.0218,2016-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14432541,-0.0050,2016-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
X_train = train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
X_test = props.drop(['parcelid'], axis=1)
# printing out the shape        
print('Shape of train dataset: {}\nShape of test dataset: {}'.format(X_train.shape, X_test.shape))

# taking out potential outliers to fit our model better
train_df=train_df[train_df.logerror > -0.4 ]
train_df=train_df[train_df.logerror < 0.42 ]
X_train=train_df.drop(['parcelid', 'logerror','transactiondate'], axis=1)
y_train = train_df["logerror"].values.astype(np.float32)
y_mean = np.mean(y_train)

Shape of train dataset: (90275, 57)
Shape of test dataset: (90274, 57)


In [5]:
preprocess(X_train, X_test) # Conducts Imputation and StandardScaler on the X_train/X_test sets

# Keras Sequential

In [6]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU, Conv1D,MaxPooling1D
from keras.layers import Dropout, BatchNormalization, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from keras import backend as K

2024-04-03 10:58:56.440780: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 10:58:56.500719: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-03 10:58:56.501276: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 10:58:57.353309: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
count = 8
len_x=int(X_train.shape[1])
print("len_x is:",len_x)

len_x is: 57


In [8]:
## The model- Uses a few dense layers on top of LSTM and CNN layers. 

model = Sequential()
model.add(Conv1D(filters=20, kernel_size=10 ,strides=10,     
                  input_shape = (len_x,1), kernel_initializer= 'uniform',      
                  activation= 'relu'))
model.add(MaxPooling1D(pool_size=2, strides=10, padding='same'))
model.add(Dropout(.17))
model.add(LSTM(units = 150 , kernel_initializer = 'normal', activation = 'softmax',return_sequences=True))
model.add(BatchNormalization())
model.add(Dropout(.4))
model.add(Dense(units = 150 , kernel_initializer = 'normal', activation = 'relu'))

model.add(BatchNormalization())
model.add(Dropout(.32))
model.add(Dense(units = 75, kernel_initializer = 'normal', activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(.22))
model.add(Dense(units = 25, kernel_initializer = 'normal', activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(.22))
model.add(Flatten())
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_absolute_error', optimizer='rmsprop')

model.summary()

2024-04-03 10:58:58.523607: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-03 10:58:58.524250: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 5, 20)             220       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 20)            0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 1, 20)             0         
                                                                 
 lstm (LSTM)                 (None, 1, 150)            102600    
                                                                 
 batch_normalization (BatchN  (None, 1, 150)           600       
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 1, 150)            0

In [9]:
print(len(X_train), len(y_train))
x = np.array(X_train)
x = np.expand_dims(x, axis=2)
y = np.array(y_train)
print(x.shape,y.shape)

88528 88528
(88528, 57, 1) (88528,)


In [10]:
model.fit(x,y, batch_size = 128, epochs = 20, verbose=1)

print("x_test.shape:",X_test.shape)
x_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

Epoch 1/20
692/692 [==============================] - 11s 11ms/step - loss: nan
Epoch 2/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 3/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 4/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 5/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 6/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 7/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 8/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 9/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 10/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 11/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 12/20
692/692 [==============================] - 8s 11ms/step - loss: nan
Epoch 13/20
692/692 [===========================

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [11]:
y_pred_ann = model.predict(X_test)
print(y_pred_ann.shape)
y_pred = y_pred_ann.flatten()
print(y_pred.shape)

2822/2822 [==============================] - 11s 4ms/step
(90274, 1)
(90274,)
